In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

**Chargement du modèle**

In [2]:
from unsloth import FastLanguageModel
import torch


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = 1048,  
    load_in_4bit = True,     
    load_in_8bit = False,    
    full_finetuning = False,
    
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-28 09:05:41.375768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751101541.624810      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751101541.673484      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.8: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

**Configuration de LoRA avec un rang réduit (r=4), sans dropout, ciblant les couches de projection pour un affinage efficace et peu coûteux.**

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,          
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8, 
    lora_dropout = 0,
    bias = "none",    
    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,   
    loftq_config = None,  
)

Unsloth 2025.6.8 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


**Chargement de la dataset**

In [4]:
from datasets import load_dataset

dataset = load_dataset("AbdelilahFdg/QA", split = "train")

QA-darija.parquet:   0%|          | 0.00/14.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65878 [00:00<?, ? examples/s]

**Division du jeu de données en ensembles d’entraînement (80 %), validation (10 %) et test (10 %), avec échantillonnage de 30 exemples pour l’évaluation rapide.**

In [5]:
train_ratio = 0.8
eval_ratio = 0.1 

# Mélange aléatoire du dataset
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(range(10000))

# Calcul des tailles
total = len(dataset)
train_size = int(total * train_ratio)
eval_size = int(total * eval_ratio)
test_size = total - train_size - eval_size

# Division
train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, train_size + eval_size))
test_dataset = dataset.select(range(train_size + eval_size, total))
eval_dataset = eval_dataset.select(range(30))

In [6]:
eval_dataset.shape

(30, 1)

**Application du template de conversation du tokenizer pour formater les dialogues en vue de l'entraînement, de la validation et du test.**

In [7]:


train_dataset = tokenizer.apply_chat_template(
    train_dataset["conversations"],
    tokenize = False,
)
eval_dataset = tokenizer.apply_chat_template(
    eval_dataset["conversations"],
    tokenize = False,
)
test_dataset = tokenizer.apply_chat_template(
    test_dataset["conversations"],
    tokenize = False,
)

In [8]:
print(train_dataset[0])

<|im_start|>user
شنو هو التفسير المحتمل للفرق الكبير بين معدل العمر عند الزواج الأول بين الرجال والنساء؟<|im_end|>
<|im_start|>assistant
<think>

</think>

خطأ ف تسجيل البيانات<|im_end|>



**Ajout d’un message système en darija en tête de chaque exemple pour guider le style des réponses générées.**

In [9]:
import pandas as pd

system_msg = "<|im_start|>system\nجاوب ديما فالدّارجة المغربية، بالحروف العربية.\n<|im_end|>\n"

train_dataset = pd.Series(train_dataset, name="text")
eval_dataset = pd.Series(eval_dataset, name="text")
test_dataset = pd.Series(test_dataset, name="text")

# Ajouter le message système au début de chaque exemple
train_dataset = train_dataset.apply(
    lambda x: system_msg + x if system_msg.strip() not in x else x
)

eval_dataset = eval_dataset.apply(
    lambda x: system_msg + x if system_msg.strip() not in x else x
)

test_dataset = test_dataset.apply(
    lambda x: system_msg + x if system_msg.strip() not in x else x
)

**Conversion des ensembles d'entraînement, validation et test en objets Dataset compatibles avec la bibliothèque Transformers.**

In [10]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(pd.DataFrame(train_dataset))
eval_dataset = Dataset.from_pandas(pd.DataFrame(eval_dataset))
test_dataset = Dataset.from_pandas(pd.DataFrame(test_dataset))

**Installation des bibliothèques nécessaires pour le calcul des métriques d’évaluation (BERTScore, ROUGE, BLEU, etc.).**

In [11]:
%%capture

!pip install evaluate
!pip install bert_score
!pip install rouge_score
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

**Configuration de la gestion mémoire CUDA pour permettre une allocation dynamique via des segments extensibles.**

In [12]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

**Fonction de calcul des métriques d'évaluation (BERTScore, BLEU, ROUGE, CHRF)**

In [13]:
import evaluate
import numpy as np
import torch

# Chargement des métriques
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
chrf = evaluate.load("chrf")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Convert logits to token IDs if necessary
    if isinstance(preds, np.ndarray) and len(preds.shape) == 3:
        preds = np.argmax(preds, axis=-1)

    # Replace -100 in labels with pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Convert to lists
    preds = preds.tolist() if isinstance(preds, np.ndarray) else preds
    labels = labels.tolist() if isinstance(labels, np.ndarray) else labels

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # BERTScore (sur CPU pour éviter OOM)
    result_bertscore = bertscore.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        lang="ar",  # langue "ar" même si c'est darija
        device="cpu"
    )

    # BLEU
    try:
        result_bleu = bleu.compute(
            predictions=decoded_preds,
            references=[[l] for l in decoded_labels]
        )
        bleu_score = result_bleu["bleu"]
    except Exception as e:
        print(f"BLEU computation failed: {e}")
        bleu_score = 0.0

    # ROUGE
    result_rouge = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    # CHRF
    result_chrf = chrf.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )

    # Compilation des métriques
    metrics = {
        "eval_bertscore_f1": np.mean(result_bertscore["f1"]),
        "eval_bleu": bleu_score,
        "eval_rougeL": result_rouge["rougeL"],
        "eval_chrf": result_chrf["score"],
    }

    # Log console
    print(
        f"Évaluation - BERTScore F1: {metrics['eval_bertscore_f1']:.4f}, "
        f"BLEU: {metrics['eval_bleu']:.4f}, "
        f"ROUGE-L: {metrics['eval_rougeL']:.4f}, "
        f"CHRF: {metrics['eval_chrf']:.4f}",
        flush=True
    )

    return metrics


**Configuration et initialisation du SFTTrainer pour fine-tuning supervisé**

In [14]:
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    compute_metrics = compute_metrics,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        per_device_eval_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 60,
        # max_steps = 30,
        num_train_epochs = 4,
        learning_rate = 2e-5,
        logging_steps = 200,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        output_dir = "/kaggle/working/checkpoints",
        save_steps = 200,
        eval_strategy = "steps",  # <-- active l'évaluation automatique
        eval_steps = 200,                
        gradient_checkpointing = True,
        max_seq_length=650,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        log_level="info",  # Ensure info-level logs are shown
        logging_strategy="steps",
        disable_tqdm=False,
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/30 [00:00<?, ? examples/s]

Using auto half precision backend


**Modifie le trainer pour entraîner uniquement sur les réponses de l'assistant**

In [15]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=4):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/30 [00:00<?, ? examples/s]

**Entrainement du modele**

In [16]:
trainer_stats = trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 4 | Total steps = 2,000
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 8,257,536/4,000,000,000 (0.21% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,Bertscore F1,Bleu,Rougel,Chrf
200,2.830700,2.246203,0.585097,0.027571,0.455502,32.477842
400,2.185300,2.083238,0.590567,0.029183,0.485718,33.489250
600,2.026100,1.979778,0.588638,0.029200,0.474186,33.733307
800,1.965500,1.898172,0.590768,0.030157,0.479987,33.991980
1000,1.883100,1.840976,0.590545,0.030580,0.486891,34.372767
1200,1.847300,1.800295,0.589709,0.030646,0.439000,34.467795
1400,1.802800,1.773094,0.588872,0.030780,0.438445,34.648487
1600,1.781300,1.749910,0.590373,0.031144,0.441646,34.661663
1800,1.779200,1.743914,0.590207,0.031111,0.483415,34.803794
2000,1.763400,1.736543,0.591879,0.031470,0.491040,34.728028


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 30
  Batch size = 2
Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

Saving model checkpoint to /kaggle/working/checkpoints/checkpoint-1600
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--qwen3-4b-unsloth-bnb-4bit/snapshots/bbe12ebf9362b3bcbf44e826e4c98910f2c02e72/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "pad_token_id": 151654,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_

Évaluation - BERTScore F1: 0.5902, BLEU: 0.0311, ROUGE-L: 0.4834, CHRF: 34.8038


Saving model checkpoint to /kaggle/working/checkpoints/checkpoint-1800
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--qwen3-4b-unsloth-bnb-4bit/snapshots/bbe12ebf9362b3bcbf44e826e4c98910f2c02e72/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "pad_token_id": 151654,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_

Évaluation - BERTScore F1: 0.5919, BLEU: 0.0315, ROUGE-L: 0.4910, CHRF: 34.7280


Saving model checkpoint to /kaggle/working/checkpoints/checkpoint-2000
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--qwen3-4b-unsloth-bnb-4bit/snapshots/bbe12ebf9362b3bcbf44e826e4c98910f2c02e72/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "pad_token_id": 151654,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_

In [2]:
print(trainer_stats)

NameError: name 'trainer_stats' is not defined

**Évaluation du modèle sur le jeu de test en utilisant les métriques définies**

In [ ]:
results = trainer.evaluate(
    eval_dataset=test_dataset,
    metric_key_prefix="test"
)

**Pousse le modèle quantifié au format GGUF vers le HF Hub avec le tokenizer associé.**

In [35]:
 # Pushing to HF Hub
 model.push_to_hub_gguf("AbdelilahFdg/darija-chatt", tokenizer, quantization_method = "q4_k_m", token = "x")

Cloning into 'llama.cpp'...
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/ggml-kompute/kompute'
Cloning into '/kaggle/working/llama.cpp/ggml/src/ggml-kompute/kompute'...
Submodule path 'ggml/src/ggml-kompute/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'
make: Entering directory '/kaggle/working/llama.cpp'
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAV

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 3.6G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 20.86 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 36/36 [00:01<00:00, 29.53it/s]
Model config Qwen3Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "max_position_embeddings": 40960,
  "max_window_layers": 36,
  "model_type": "qwen3",
  "num_attention_heads": 32,
  "num_hidden_layers": 36,
  "num_key_value_heads": 8,
  "pad_token_id": 151654,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "float16",
  "transformers_version": "4.51.3",
  "unsloth_fixed": true,
  "unsloth_version": "2025.5.9",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Unsloth: Saving tokenizer... Done.


...

Unsloth: Converting qwen3 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at AbdelilahFdg/darija-chatt into f16 GGUF format.
The output location will be /kaggle/working/AbdelilahFdg/darija-chatt/unsloth.F16.gguf
This might take 3 minutes...
Writing: 100%|██████████| 8.05G/8.05G [00:22<00:00, 354Mbyte/s] 
Unsloth: Conversion completed! Output location: /kaggle/working/AbdelilahFdg/darija-chatt/unsloth.F16.gguf
Unsloth: [2] Converting GGUF 16bit into q4_k_m. This might take 20 minutes...
main: build = 5599 (1caae7fc)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing '/kaggle/working/Abde

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/2.50G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/AbdelilahFdg/darija-chatt
